In [4]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t
import moyra as ma
me.mechanics_printing()

In [5]:
# define Degrees of freedom
bending = 1
in_plane_bend = 0
torsion = 1
inner_dofs = bending + in_plane_bend + torsion
DoFs = (inner_dofs + 1)*2 + 1
# DoFs = inner_dofs

In [6]:
# generate parameters
p = ma.DynamicModelParameters(DoFs)

#gravity vector
p.DoFs = ma.ModelSymbol(value = DoFs,string = 'DoFs')                     # DoFs
p.g  = ma.ModelSymbol(value = 9.81,string = 'g')                     # gravity
p.g_v = ma.ModelVector(value =[0,0,-1],length=3,string='g_v') # The stationary point

# main geometric properties
p.y_h = ma.ModelSymbol(value = 0.3,string = 'y_h')                  # y position of hinge line along beam axis
p.Lambda = ma.ModelSymbol(value = 0.3,string = 'Lambda')            # Flare Angle

# inner beam properties
p.b_i = ma.ModelSymbol(value = 0.3,string = 'b_i')                  # y position beam end
p.rho_b = ma.ModelSymbol(value = 0.3,string = 'rho_b')              # density per unit area      
p.c_b = ma.ModelSymbol(value = 0.3,string = 'c_b')                  # beam chord
p.EI = ma.ModelSymbol(value = 0.3,string = 'EI')                    # beam flexural rigidity
p.GJ = ma.ModelSymbol(value = 0.3,string = 'GJ')                    # beam torsional rigidity
p.K = ma.ModelSymbol(value = 0.3,string = 'K')                      # beam Cross Coupling

# inner Section mass properties
p.m_s = ma.ModelSymbol(value = 0,string = 'm_s')                    # Sectional Mass
p.I_s = ma.ModelVector(length=3,string='I_s')        # Moments of Inertia for Section 
p.x_s = ma.ModelSymbol(value = 0.3,string = 'x_s')                  # x postion of the CoM of each section 
p.y_s = ma.ModelVector(length=3,string='y_s')        # y position of each of the 3 sections

# # Aileron servo mass properties
# p.m_sv = ma.ModelSymbol(value = 0,string = 'm_sv')                  # Servo Mass
# p.X_sv = ma.ModelMatrix(value =[1,1,0],length=3,string='X_sv')      # Servo x,y,z postion (z ignored)

# inner Hinge mass properties
p.m_h = ma.ModelSymbol(value = 0,string = 'm_h')                    # additional hinge mass section
p.I_h = ma.ModelVector(value =[0,0,0],length=3,string='I_h')        # Moments of inertia of additonal hinge mass
p.X_h = ma.ModelVector(value =[1,1,0],length=3,string='X_h')        # location of addtional hinge mass (z ignored)
p.k_h = ma.ModelSymbol(value = 0,string = 'k_h')                    # Hinge Stiffness
p.d_h = ma.ModelSymbol(value = 0,string = 'd_h')                    # Hinge Damping

# FWT mass properties
p.m_f = ma.ModelSymbol(value = 0,string = 'm_f')                    # FWT mass
p.I_f = ma.ModelVector(value =[0,0,0],length=3,string='I_f')        # FWT moments of inertia
p.X_f = ma.ModelVector(value =[1,1,0],length=3,string='X_f')        # location of FWT CoM relative to hinge semi-chord

# Xb = ma.ModelVector(value =[1,1,0],length=3,string='Xb')
# Xb[0],Xb[1],p.Gamma_0 = sym.symbols('x_0,y_0,Gamma_0')
p.Gamma_0 = sym.symbols('Gamma_0')
Xb = ma.VarVector('Xb',3,1)

In [7]:
## GET INNER WING ELEMENTS
# get shape function for RHS wing
Xr = Xb/p.b_i
q_f_RHS = p.q[1:inner_dofs+1]
S,tau_RHS = ma.elements.FlexiElement.ShapeFunctions_OBM_IBN_TO(bending,in_plane_bend,torsion,q_f_RHS,Xr[1],Xr[0],0,1,type='taylor')
u = S*sym.Matrix(q_f_RHS) + sym.Matrix([Xb[0],Xb[1],0])
body_frame = ma.frames.HomogenousFrame().R_x(p.q[0])
inner_wing_frame_RHS = body_frame.Translate(*u)

# get shape function for LHS wing
Xl = Xb/p.b_i
q_f_LHS = p.q[inner_dofs+2:inner_dofs*2+2]
S,tau_LHS= ma.elements.FlexiElement.ShapeFunctions_OBM_IBN_TO(bending,in_plane_bend,torsion,q_f_LHS,Xl[1],Xl[0],0,1,type='taylor')
u = S*sym.Matrix(q_f_LHS) + sym.Matrix([Xb[0],Xb[1],0])
inner_wing_frame_LHS = body_frame.Translate(*u)

## create inner wing elements
inner_elements = []
# get beam element RHS
link2_x__int = (Xb[0],-p.c_b*sym.Rational(1,2),p.c_b*sym.Rational(1,2))
link2_y_int = (Xb[1],0,p.b_i)
main_wing_ele_RHS = ma.elements.FlexiElement(p.q,inner_wing_frame_RHS,p.rho_b,S,link2_x__int,link2_y_int,0,q_f_RHS,p.EI,p.GJ,grav_vec = p.g*p.g_v,simplify = False, name="Inner Beam",K=p.K)
inner_elements.append(main_wing_ele_RHS)

# get wing section elements RHS
M_s = ma.elements.MassMatrix(p.m_s,*p.I_s)
for i in range(len(p.y_s)):
    tmp_frame = inner_wing_frame_RHS.R_y(tau_RHS).subs({Xb[0]:p.x_s,Xb[1]:p.y_s[i]})
    inner_elements.append(ma.elements.RigidElement(p.q,tmp_frame,M_s,grav_vec = p.g*p.g_v, simplify = False, name=f"Wing Section {i+1} RHS"))

# get hinge mass element RHS
M = ma.elements.MassMatrix(p.m_h,*p.I_h)
tmp_frame = inner_wing_frame_RHS.R_y(tau_RHS).subs({Xb[0]:p.X_h[0],Xb[1]:p.X_h[1]})
inner_elements.append(ma.elements.RigidElement(p.q,tmp_frame,M,grav_vec = p.g*p.g_v, simplify = False, name="Hinge Mass Element RHS"))    

# get beam element LHS
link2_x__int = (Xb[0],-p.c_b*sym.Rational(1,2),p.c_b*sym.Rational(1,2))
link2_y_int = (Xb[1],0,-p.b_i)
main_wing_ele_LHS = ma.elements.FlexiElement(p.q,inner_wing_frame_LHS,p.rho_b,S,link2_x__int,link2_y_int,0,q_f_LHS,p.EI,p.GJ,grav_vec = p.g*p.g_v,simplify = False, name="Inner Beam",K=p.K)
inner_elements.append(main_wing_ele_LHS)

# get wing section elements LHS
M_s = ma.elements.MassMatrix(p.m_s,*p.I_s)
for i in range(len(p.y_s)):
    tmp_frame = inner_wing_frame_LHS.R_y(tau_LHS).subs({Xb[0]:p.x_s,Xb[1]:-p.y_s[i]})
    inner_elements.append(ma.elements.RigidElement(p.q,tmp_frame,M_s,grav_vec = p.g*p.g_v, simplify = False, name=f"Wing Section {i+1} LHS"))

# get hinge mass element LHS
M = ma.elements.MassMatrix(p.m_h,*p.I_h)
tmp_frame = inner_wing_frame_LHS.R_y(tau_LHS).subs({Xb[0]:p.X_h[0],Xb[1]:-p.X_h[1]})
inner_elements.append(ma.elements.RigidElement(p.q,tmp_frame,M,grav_vec = p.g*p.g_v, simplify = False, name="Hinge Mass Element LHS"))

In [9]:
p.q[inner_dofs+1]

q₃

In [ ]:
## GET FWT ELEMENT RHS

# get angles at end of inner beam
dz_dy_RHS = u.subs({Xb[0]:0}).diff(Xb[1]).subs({Xb[1]:p.y_h})
dz_dx_RHS = u.subs({Xb[1]:p.y_h}).diff(Xb[0]).subs({Xb[0]:0})
Ry_RHS = sym.Function('Ry_r')(t)
Rx_RHS = sym.Function('Rx_r')(t)

# get fwt frames
hinge_rot_RHS = ma.frames.HomogenousFrame().R_y(Ry_RHS).R_x(Rx_RHS).R_z(p.Lambda).simplify().A
fold_rot_RHS = ma.frames.HomogenousFrame().R_x(-p.q[inner_dofs+1]).R_z(-p.Lambda).simplify().A

hinge_frame_RHS = inner_wing_frame_RHS.subs({Xb[1]:p.y_h,Xb[0]:0}).Rotate(hinge_rot_RHS).simplify()
fwt_base_frame_RHS = hinge_frame_RHS.Rotate(fold_rot_RHS).simplify()
fwt_frame_RHS = fwt_base_frame_RHS.Translate(*p.X_f)

#create fwt main elements
fwt_elements = []
M = ma.elements.MassMatrix(p.m_f,p.I_f[0],p.I_f[1],p.I_f[2])
# M = ma.elements.MassMatrix(p.m_f,0,0,0)
fwt_elements.append(ma.elements.RigidElement(p.q,fwt_frame_RHS,M,grav_vec = p.g*p.g_v, simplify = False,name="FWT RHS"))

# get hinge spring and damper elements
fwt_elements.append(ma.elements.Spring(p.q,p.q[inner_dofs+1+1],p.k_h,name="Hinge Spring RHS"))
fwt_elements.append(ma.elements.Damper(p.q,p.q[inner_dofs+1+1],p.d_h,name="Hinge Damper RHS"))

## GET FWT ELEMENT LHS

# get angles at end of inner beam
dz_dy_LHS = u.subs({Xb[0]:0}).diff(Xb[1]).subs({Xb[1]:-p.y_h})
dz_dx_LHS = u.subs({Xb[1]:p.y_h}).diff(Xb[0]).subs({Xb[0]:0})
Ry_LHS = sym.Function('Ry_l')(t)
Rx_LHS = sym.Function('Rx_l')(t)

# get fwt frames
hinge_rot_LHS = ma.frames.HomogenousFrame().R_y(Ry_LHS).R_x(Rx_LHS).R_z(-p.Lambda).simplify().A
fold_rot_LHS = ma.frames.HomogenousFrame().R_x(-p.q[-1]).R_z(p.Lambda).simplify().A

hinge_frame_LHS = inner_wing_frame_LHS.subs({Xb[1]:-p.y_h,Xb[0]:0}).Rotate(hinge_rot_LHS).simplify()
fwt_base_frame_LHS = hinge_frame_LHS.Rotate(fold_rot_LHS).simplify()
fwt_frame_LHS = fwt_base_frame_LHS.Translate(*p.X_f)

#create fwt main elements
M = ma.elements.MassMatrix(p.m_f,p.I_f[0],p.I_f[1],p.I_f[2])
# M = ma.elements.MassMatrix(p.m_f,0,0,0)
fwt_elements.append(ma.elements.RigidElement(p.q,fwt_frame_LHS,M,grav_vec = p.g*p.g_v, simplify = False,name="FWT LHS"))

# get hinge spring and damper elements
fwt_elements.append(ma.elements.Spring(p.q,p.q[-1],p.k_h,name="Hinge Spring LHS"))
fwt_elements.append(ma.elements.Damper(p.q,p.q[-1],p.d_h,name="Hinge Damper LHS"))

In [6]:
# sm = ma.SymbolicModel.FromElementsAndForces(p.q,[*inner_elements,*fwt_elements])
# Xi = ma.VarVector('Xi',3)


# subs = {p.Gamma_0:dz_dy[2]/dz_dy[1]}

subs = {Rx_RHS.diff(t):(dz_dy_RHS[2]/dz_dy_RHS[1]).diff(t),Ry_RHS.diff(t):(-dz_dx_RHS[2]/dz_dx_RHS[0]).diff(t),
          Rx_RHS:dz_dy_RHS[2]/dz_dy_RHS[1],Ry_RHS:-dz_dx_RHS[2]/dz_dx_RHS[0],
          Rx_LHS.diff(t):(dz_dy_LHS[2]/dz_dy_LHS[1]).diff(t),Ry_LHS.diff(t):(-dz_dx_LHS[2]/dz_dx_LHS[0]).diff(t),
          Rx_LHS:dz_dy_LHS[2]/dz_dy_LHS[1],Ry_LHS:-dz_dx_LHS[2]/dz_dx_LHS[0]}
# subs = {Rx.diff(t):0,Ry.diff(t):0,
#           Rx:dz_dy[2]/dz_dy[1],Ry:-dz_dx[2]/dz_dx[0]}

sm = ma.SymbolicModel.FromElementsAndForces(p.q,[*inner_elements,*fwt_elements]).subs(subs)
# sm = ma.SymbolicModel.FromElementsAndForces(p.q,[inner_elements[0]]).subs(subs)
fwt_frame_RHS = fwt_frame_RHS.subs(subs)
fwt_base_frame_RHS = fwt_base_frame_RHS.subs(subs)
fwt_frame_LHS = fwt_frame_LHS.subs(subs)
fwt_base_frame_LHS = fwt_base_frame_LHS.subs(subs)
# %lprun -f ma.SymbolicModel.FromElementsAndForces ma.SymbolicModel.FromElementsAndForces(p.q,[main_wing_ele,fwt_ele])

Generating EoM for Element 1 out of 16 - Inner Beam:FlexiElement
Generating EoM for Element 2 out of 16 - Wing Section 1 RHS:RigidElement
Generating EoM for Element 3 out of 16 - Wing Section 2 RHS:RigidElement
Generating EoM for Element 4 out of 16 - Wing Section 3 RHS:RigidElement
Generating EoM for Element 5 out of 16 - Hinge Mass Element RHS:RigidElement
Generating EoM for Element 6 out of 16 - Inner Beam:FlexiElement
Generating EoM for Element 7 out of 16 - Wing Section 1 LHS:RigidElement
Generating EoM for Element 8 out of 16 - Wing Section 2 LHS:RigidElement
Generating EoM for Element 9 out of 16 - Wing Section 3 LHS:RigidElement
Generating EoM for Element 10 out of 16 - Hinge Mass Element LHS:RigidElement
Generating EoM for Element 11 out of 16 - FWT RHS:RigidElement
Generating EoM for Element 12 out of 16 - Hinge Spring RHS:Spring
Generating EoM for Element 13 out of 16 - Hinge Damper RHS:Damper
Generating EoM for Element 14 out of 16 - FWT LHS:RigidElement
Generating EoM for 

In [7]:
funcs = []
Xi = ma.VarVector('Xi',3,1)
funcs.append(('get_pos_inner_wing_RHS',inner_wing_frame_RHS.transform_point([0]*3)))
funcs.append(('get_pos_fwt_RHS',fwt_frame_RHS.transform_point(Xi)))
funcs.append(('get_fwt_V_b_RHS',fwt_base_frame_RHS.A.T*fwt_base_frame_RHS.transform_point(Xi).diff(t)))
funcs.append(('get_fwt_V_global_RHS',fwt_base_frame_RHS.transform_point(Xi).diff(t)))
funcs.append(('get_pos_global2fwt_RHS',fwt_base_frame_RHS.transform_global_point(Xi)))
funcs.append(('get_pos_fwt2global_RHS',fwt_base_frame_RHS.transform_point(Xi)))
funcs.append(('get_fwt_V_b_body_RHS',body_frame.A.T*fwt_base_frame_RHS.transform_point(Xi).diff(t)))
funcs.append(('get_hda_RHS',dz_dy_RHS[2]/dz_dy_RHS[1]))
funcs.append(('get_fwt_A_RHS',fwt_frame_RHS.A))
funcs.append(('get_inner_A_RHS',inner_wing_frame_RHS.A))
funcs.append(('get_inner_V_b_RHS',inner_wing_frame_RHS.A.T*inner_wing_frame_RHS.R.diff(t)))

funcs.append(('get_pos_inner_wing_LHS',inner_wing_frame_LHS.transform_point([0]*3)))
funcs.append(('get_pos_fwt_LHS',fwt_frame_LHS.transform_point(Xi)))
funcs.append(('get_fwt_V_b_LHS',fwt_base_frame_LHS.A.T*fwt_base_frame_LHS.transform_point(Xi).diff(t)))
funcs.append(('get_fwt_V_global_LHS',fwt_base_frame_LHS.transform_point(Xi).diff(t)))
funcs.append(('get_pos_global2fwt_LHS',fwt_base_frame_LHS.transform_global_point(Xi)))
funcs.append(('get_pos_fwt2global_LHS',fwt_base_frame_LHS.transform_point(Xi)))
funcs.append(('get_fwt_V_b_body_LHS',body_frame.A.T*fwt_base_frame_LHS.transform_point(Xi).diff(t)))
funcs.append(('get_hda_LHS',dz_dy_LHS[2]/dz_dy_LHS[1]))
funcs.append(('get_fwt_A_LHS',fwt_frame_LHS.A))
funcs.append(('get_inner_A_LHS',inner_wing_frame_LHS.A))
funcs.append(('get_inner_V_b_LHS',inner_wing_frame_LHS.A.T*inner_wing_frame_LHS.R.diff(t)))

funcs.append(('get_body_A',body_frame.A))

In [8]:

## FWT RHS
fwt_normal_RHS = fwt_frame_RHS.transform_vector([0,0,1])
fwt_hinge_vector_RHS = fwt_frame_RHS.transform_vector([sym.cos(p.Lambda),sym.sin(p.Lambda),0])
surf_RHS = inner_wing_frame_RHS.transform_point([0]*3)
v_RHS = ma.Wedge(surf_RHS.diff(Xb[0]))*surf_RHS.diff(Xb[1])
hinge_normal_RHS = sym.simplify(v_RHS.subs({Xb[0]:0,Xb[1]:p.y_h}))

funcs.append(('get_surf_x_RHS',surf_RHS.subs({Xb[0]:1})-surf_RHS.subs({Xb[0]:0})))
funcs.append(('get_surf_z_RHS',sym.simplify(v_RHS)))
funcs.append(('get_hinge_normal_RHS',hinge_normal_RHS))
funcs.append(('get_fwt_normal_RHS',fwt_normal_RHS))
funcs.append(('get_fwt_hinge_vector_RHS',fwt_hinge_vector_RHS))


## FWT LHS
fwt_normal_LHS = fwt_frame_LHS.transform_vector([0,0,1])
fwt_hinge_vector_LHS = fwt_frame_LHS.transform_vector([sym.cos(p.Lambda),sym.sin(p.Lambda),0])
surf_LHS = inner_wing_frame_LHS.transform_point([0]*3)
v_LHS = ma.Wedge(surf_LHS.diff(Xb[0]))*surf_LHS.diff(Xb[1])
hinge_normal_LHS = sym.simplify(v_LHS.subs({Xb[0]:0,Xb[1]:-p.y_h}))

funcs.append(('get_surf_x_LHS',surf_LHS.subs({Xb[0]:1})-surf_LHS.subs({Xb[0]:0})))
funcs.append(('get_surf_z_LHS',sym.simplify(v_LHS)))
funcs.append(('get_hinge_normal_LHS',hinge_normal_LHS))
funcs.append(('get_fwt_normal_LHS',fwt_normal_LHS))
funcs.append(('get_fwt_hinge_vector_LHS',fwt_hinge_vector_LHS))

In [9]:
# generate generic force symbols
F_x,F_y,F_z,M_x,M_y,M_z = sym.symbols('F_x,F_y,F_z,M_x,M_y,M_z')
W_n = sym.Matrix([F_x,F_y,F_z,M_x,M_y,M_z])
W = ma.VarVector('W',6,1)

#create function for an arbitary force on FWT RHS
fwt_Q = fwt_base_frame_RHS.Translate(*Xi).BodyJacobian(p.q).T*W_n
fwt_Q = me.msubs(fwt_Q,{**{W_n[i]:W[i,0] for i in range(6)}})
funcs.append(('get_fwt_Q_RHS',me.msubs(fwt_Q,{p.q[0]:0,p.qd[0]:0}))) # roll does not affect the generalised force

#create function for an arbitary force on Main Wing RHS
MW_Q = inner_wing_frame_RHS.BodyJacobian(p.q).T*W_n
MW_Q = me.msubs(MW_Q,{**{W_n[i]:W[i,0] for i in range(6)}})
funcs.append(('get_inner_Q_RHS',me.msubs(MW_Q,{p.q[0]:0,p.qd[0]:0}))) # roll does not affect the generalised force

#create function for an arbitary force on FWT LHS
fwt_Q = fwt_base_frame_LHS.Translate(*Xi).BodyJacobian(p.q).T*W_n
fwt_Q = me.msubs(fwt_Q,{**{W_n[i]:W[i,0] for i in range(6)}})
funcs.append(('get_fwt_Q_LHS',me.msubs(fwt_Q,{p.q[0]:0,p.qd[0]:0}))) # roll does not affect the generalised force

#create function for an arbitary force on Main Wing LHS
MW_Q = inner_wing_frame_LHS.BodyJacobian(p.q).T*W_n
MW_Q = me.msubs(MW_Q,{**{W_n[i]:W[i,0] for i in range(6)}})
funcs.append(('get_inner_Q_LHS',me.msubs(MW_Q,{p.q[0]:0,p.qd[0]:0}))) # roll does not affect the generalised force

In [10]:
base_dir = "C:/git/roll_flexibility/src_matlab"
class_name = f'Roll_B{bending}T{torsion}_FWT_RC2'
base_class = "mbd.BaseRC"
sm.to_matlab_class(p,base_dir,class_name,base_class,additional_funcs=funcs)

Unknown variable Xb found in function get_pos_inner_wing_RHS. It will be added to the function signature.
Unknown variable Xi found in function get_pos_fwt_RHS. It will be added to the function signature.
Unknown variable Xi found in function get_fwt_V_b_RHS. It will be added to the function signature.
Unknown variable Xi found in function get_fwt_V_global_RHS. It will be added to the function signature.
Unknown variable Xi found in function get_pos_global2fwt_RHS. It will be added to the function signature.
Unknown variable Xi found in function get_pos_fwt2global_RHS. It will be added to the function signature.
Unknown variable Xi found in function get_fwt_V_b_body_RHS. It will be added to the function signature.
Unknown variable Xb found in function get_inner_V_b_RHS. It will be added to the function signature.
Unknown variable Xb found in function get_pos_inner_wing_LHS. It will be added to the function signature.
Unknown variable Xi found in function get_pos_fwt_LHS. It will be add